<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/Parquet_Files_Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2024-04-30 13:19:39.449252+05:30
Python 3.10.12
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.3 LTS
Release:	22.04
Codename:	jammy


In [2]:
import numpy as np
import pandas as pd
import pyarrow.csv as pv
import pyarrow.parquet as pq

#SuperStore Data
Super Store Order Data https://drive.google.com/file/d/13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ/view?usp=sharing <br>
Original Data from [Tableau](https://community.tableau.com/docs/DOC-1236)

In [3]:
#!wget -O SS_Orders2.csv 'https://drive.google.com/uc?export=download&id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ'
#!gdown https://drive.google.com/uc?id=13MKOpo8iD-tMkeMLw-4IPFi5-xWhmqFZ
#!wget -q -O Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'
!wget -q -O Orders.zip 'https://github.com/Praxis-QR/PythonForManagers/raw/main/Data/SuperStoreSalesKaggle.zip'

In [ ]:
#!tail Orders.csv

#Basic Pandas Operations

In [4]:
#df = pd.read_csv('Orders.csv', delimiter = ',')
df = pd.read_csv('Orders.zip', delimiter = ',')
df.head(2)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011


In [5]:
# order_date, ship_date, sales have been incorrectly formatted as string
# year has been incorrectly formatted as integer
df.info()
#df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  object 
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

##Comma, Double Quotes look good

In [6]:
#df.iloc[9993]['Product Name']
df.iloc[9993]['product_name']

'Boston Canvas, Water Color'

In [7]:
#df.iloc[9992]['Product Name']
df.iloc[9992]['product_name']

'Hon Racetrack Conference Tables'

## Sales, year needs to be fixed

In [8]:
#
# df.sales has been automatically interpreted as string because of the presence of commma,
# so replace comma and convert to numeric
#

df['sales'] = df['sales'].str.replace(',', '')    # replace , with blank
df['sales'] = pd.to_numeric(df['sales'])          # convert column to numeric
df['year'] = df['year'].astype(str)          # convert column to string

In [ ]:
#df.info()

##Date looks Bad, need to be fixed

In [24]:
# what is the first and last order date ?
# we have four years of data, but date format changes ...
df['order_date'].head(), df['order_date'].tail()

(0    1/1/2011
 1    1/1/2011
 2    1/1/2011
 3    1/1/2011
 4    1/1/2011
 Name: order_date, dtype: object,
 51285    31-12-2014
 51286    31-12-2014
 51287    31-12-2014
 51288    31-12-2014
 51289    31-12-2014
 Name: order_date, dtype: object)

In [25]:
# Some of the dates are string like dd/mm/yyyy while others are dd-mm-yyyy
# this will cause problems in converting string to real dates
# So we move all dd-mm-yyyy to dd/mm/yyyy -- could have been the other way round also
#
#df['Order Date'] = df['Order Date'].str.replace('-','/')
#df['Ship Date'] = df['Ship Date'].str.replace('-','/')

df['order_date'] = df['order_date'].str.replace('-','/')
df['ship_date'] = df['ship_date'].str.replace('-','/')

In [26]:
# convert all dates to dd/mm/yyyy format
#
df['ship_date'] = pd.to_datetime(df['ship_date'],format='%d/%m/%Y')
df['order_date'] = pd.to_datetime(df['order_date'],format='%d/%m/%Y')
#df['order_date'].describe

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  int64         
 15  qu

In [28]:
df.to_csv('Orders2.csv', index=False)

##Pandas Pivot Tables
https://github.com/Praxis-QR/PythonForManagers/blob/main/Retail_Sales_Data_Analysis.ipynb

In [29]:
#Create the pivot table
CrossTab = pd.pivot_table(data=df,index='region',columns='segment',values='sales', aggfunc='sum', margins=True).copy()
CrossTab

segment,Consumer,Corporate,Home Office,All
region,,,,
Africa,423751,204948,155077,783776
Canada,35717,19319,11896,66932
Caribbean,162351,104535,57395,324281
Central,1480033,850405,491961,2822399
Central Asia,369052,235552,148235,752839
EMEA,406763,250559,148862,806184
East,350947,200422,127465,678834
North,643976,394220,209996,1248192
North Asia,407410,265762,175177,848349


In [30]:
#Create the pivot table
CrossTab = pd.pivot_table(data=df,index='category',columns='year',values='sales', aggfunc='sum', margins=True).copy()
CrossTab

year,2011,2012,2013,2014,All
category,,,,,
Furniture,756171,858913,1117739,1378061,4110884
Office Supplies,675642,795113,1010793,1305782,3787330
Technology,827698,1023467,1277328,1616198,4744691
All,2259511,2677493,3405860,4300041,12642905


#Pandas Dataframe to Parquet Format
Ref https://mungingdata.com/pyarrow/parquet-metadata-min-max-statistics/

In [31]:
df.to_parquet('Orders2.parquet')

In [32]:
# Note that parquet files are much smaller even compared to the zip file
#!ls -al Orders2.*
!ls -al

total 14404
drwxr-xr-x 1 root root     4096 Apr 30 08:04 .
drwxr-xr-x 1 root root     4096 Apr 30 07:48 ..
drwxr-xr-x 4 root root     4096 Apr 26 13:21 .config
-rw-r--r-- 1 root root 10865023 Apr 30 08:03 Orders2.csv
-rw-r--r-- 1 root root  1579365 Apr 30 08:04 Orders2.parquet
-rw-r--r-- 1 root root  2283831 Apr 30 07:50 Orders.zip
drwxr-xr-x 1 root root     4096 Apr 26 13:22 sample_data


In [33]:
import pyarrow.csv as pv
import pyarrow.parquet as pq

In [34]:
pf = pq.ParquetFile('Orders2.parquet')

In [35]:
pf.metadata

  created_by: parquet-cpp-arrow version 14.0.2
  num_columns: 21
  num_rows: 51290
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 10815

In [36]:
pf.metadata.row_group(0)

  num_columns: 21
  num_rows: 51290
  total_byte_size: 2104641

In [37]:
# Get details of each column
pf.metadata.row_group(0).column(1)

  file_offset: 310878
  file_path: 
  physical_type: INT64
  num_values: 51290
  path_in_schema: order_date
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 2011-01-01 00:00:00
      max: 2014-12-31 00:00:00
      null_count: 0
      distinct_count: None
      num_values: 51290
      physical_type: INT64
      logical_type: Timestamp(isAdjustedToUTC=false, timeUnit=nanoseconds, is_from_converted_type=false, force_set_converted_type=false)
      converted_type (legacy): NONE
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 294520
  data_page_offset: 304995
  total_compressed_size: 16358
  total_uncompressed_size: 17356

In [38]:
# Get details of each column
pf.metadata.row_group(0).column(5)

  file_offset: 491990
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 51290
  path_in_schema: segment
  is_stats_set: True
  statistics:
      has_min_max: True
      min: Consumer
      max: Home Office
      null_count: 0
      distinct_count: None
      num_values: 51290
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 478942
  data_page_offset: 478998
  total_compressed_size: 13048
  total_uncompressed_size: 13041

In [ ]:
pf.metadata.row_group(0).column(20)

  file_offset: 1568466
  file_path: 
  physical_type: BYTE_ARRAY
  num_values: 51290
  path_in_schema: year
  is_stats_set: True
  statistics:
      has_min_max: True
      min: 2011
      max: 2014
      null_count: 0
      distinct_count: None
      num_values: 51290
      physical_type: BYTE_ARRAY
      logical_type: String
      converted_type (legacy): UTF8
  compression: SNAPPY
  encodings: ('PLAIN', 'RLE', 'RLE_DICTIONARY')
  has_dictionary_page: True
  dictionary_page_offset: 1568364
  data_page_offset: 1568404
  total_compressed_size: 102
  total_uncompressed_size: 106

#Parquet to Pandas Dataframe

In [40]:
dfFull = pd.read_parquet('Orders2.parquet')
dfFull.head(2)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,...,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,2011-01-01,2011-01-06,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,...,Office Supplies,Storage,"Tenex Lockers, Blue",408,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,...,Office Supplies,Supplies,"Acme Trimmer, High Speed",120,3,0.1,36.036,9.72,Medium,2011


In [41]:
# FOUR dimensions, THREE facts
#selCols = ['Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit']
selCols = ['year','segment','region','category','sales','quantity','profit']
dfDimFact = pd.read_parquet('Orders2.parquet', columns=selCols)
dfDimFact.head(2)

,year,segment,region,category,sales,quantity,profit
0,2011,Consumer,Africa,Office Supplies,408,2,106.140
1,2011,Consumer,Oceania,Office Supplies,120,3,36.036


In [42]:
dfDimFact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      51290 non-null  object 
 1   segment   51290 non-null  object 
 2   region    51290 non-null  object 
 3   category  51290 non-null  object 
 4   sales     51290 non-null  int64  
 5   quantity  51290 non-null  int64  
 6   profit    51290 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


##Parquet preserves column properties

In [43]:
# Data Still not clean in csv
#
df5A = pd.read_csv('Orders2.csv')
#df9.dtypes
df5A.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  int64  
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

In [44]:
#Data Clean with Parquet
#
df5B = pd.read_parquet('Orders2.parquet')
#df9.dtypes
df5B.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  int64         
 15  qu

#Retrieving by Row Criteria

In [ ]:
#df

In [45]:
#Note how the directory structure is created
# name of directory is 'analytics'
df.to_parquet(
    path='analytics',
    engine='pyarrow',
    compression='snappy',
    partition_cols=['segment', 'region','category','year']
 )

In [46]:
df3 = pd.read_parquet(
    path='analytics',
    engine='pyarrow',
    columns=['category', 'sales', 'quantity','profit'],
    filters=[('category', '=', 'Furniture')]
)
df3

,category,sales,quantity,profit
0,Furniture,2479,6,49.5000
1,Furniture,45,1,-56.8560
2,Furniture,51,2,22.8000
3,Furniture,436,1,178.7400
4,Furniture,334,2,26.7000
...,...,...,...,...
9871,Furniture,148,3,15.6978
9872,Furniture,26,3,11.0250
9873,Furniture,23,3,9.7911
9874,Furniture,842,3,105.2970


#PyArrow Table

In [47]:
table1 = pv.read_csv('Orders2.csv')
table2 = pq.read_table('Orders2.parquet')

In [48]:
# Date is still not clean
table1_df = table1.to_pandas()
table1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   order_id        51290 non-null  object 
 1   order_date      51290 non-null  object 
 2   ship_date       51290 non-null  object 
 3   ship_mode       51290 non-null  object 
 4   customer_name   51290 non-null  object 
 5   segment         51290 non-null  object 
 6   state           51290 non-null  object 
 7   country         51290 non-null  object 
 8   market          51290 non-null  object 
 9   region          51290 non-null  object 
 10  product_id      51290 non-null  object 
 11  category        51290 non-null  object 
 12  sub_category    51290 non-null  object 
 13  product_name    51290 non-null  object 
 14  sales           51290 non-null  int64  
 15  quantity        51290 non-null  int64  
 16  discount        51290 non-null  float64
 17  profit          51290 non-null 

In [49]:
# Data is clean
table2_df = table2.to_pandas()
table2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        51290 non-null  object        
 1   order_date      51290 non-null  datetime64[ns]
 2   ship_date       51290 non-null  datetime64[ns]
 3   ship_mode       51290 non-null  object        
 4   customer_name   51290 non-null  object        
 5   segment         51290 non-null  object        
 6   state           51290 non-null  object        
 7   country         51290 non-null  object        
 8   market          51290 non-null  object        
 9   region          51290 non-null  object        
 10  product_id      51290 non-null  object        
 11  category        51290 non-null  object        
 12  sub_category    51290 non-null  object        
 13  product_name    51290 non-null  object        
 14  sales           51290 non-null  int64         
 15  qu

#Spark
https://davidadrian.cc/python-hadoop/

In [50]:
!pip3 -q install pyspark
from pyspark.sql import SparkSession

from pyspark.sql.types import *
from pyspark.sql.functions import *
#spark9 = SparkSession.builder.master("local[*]").getOrCreate()
# note UI port switched from default 4040 to 4050 to avoid clash with ngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [51]:
# Note Extra parameter introduced in spark
# Without this there will be errors in reading Parquet files

#spark9 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
spark9 = SparkSession.\
    builder.master("local[*]").\
    config('spark.ui.port', '4050').\
    config("spark.sql.legacy.parquet.nanosAsLong", "true").\
    getOrCreate()

In [ ]:
#!tail Orders2.csv
#!tail Orders2.parquet --- unreadable

In [112]:
# Inferred INCORRECTLY, from CSV file
# Numerics read as string
# date read correctly

sdf9A = spark9.read.csv('Orders2.csv', inferSchema=True,header = True, quote = '"')
sdf9A.dtypes

[('order_id', 'string'),
 ('order_date', 'date'),
 ('ship_date', 'date'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'string'),
 ('quantity', 'string'),
 ('discount', 'string'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [ ]:
#Inferred CORRECTLY, from parquet file
#sdf9B = spark19.read.parquet('Orders2.parquet', inferSchema=True,header = True, quote = '"')
#sdf9B.dtypes

In [113]:
#Inferred CORRECTLY, from parquet file
# See Order Date / Shipdate
sdf9B = spark9.read.parquet('Orders2.parquet', inferSchema=True,header = True, quote = '"')
sdf9B.dtypes

[('order_id', 'string'),
 ('order_date', 'bigint'),
 ('ship_date', 'bigint'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [114]:
sdf9B.head()

Row(order_id='AG-2011-2040', order_date=1293840000000000000, ship_date=1294272000000000000, ship_mode='Standard Class', customer_name='Toby Braunhardt', segment='Consumer', state='Constantine', country='Algeria', market='Africa', region='Africa', product_id='OFF-TEN-10000025', category='Office Supplies', sub_category='Storage', product_name='Tenex Lockers, Blue', sales=408, quantity=2, discount=0.0, profit=106.14, shipping_cost=35.46, order_priority='Medium', year='2011')

In [115]:
sdf9B.show(3)

+-------------+-------------------+-------------------+--------------+---------------+--------+---------------+---------+------+-------+----------------+---------------+------------+--------------------+-----+--------+--------+------+-------------+--------------+----+
|     order_id|         order_date|          ship_date|     ship_mode|  customer_name| segment|          state|  country|market| region|      product_id|       category|sub_category|        product_name|sales|quantity|discount|profit|shipping_cost|order_priority|year|
+-------------+-------------------+-------------------+--------------+---------------+--------+---------------+---------+------+-------+----------------+---------------+------------+--------------------+-----+--------+--------+------+-------------+--------------+----+
| AG-2011-2040|1293840000000000000|1294272000000000000|Standard Class|Toby Braunhardt|Consumer|    Constantine|  Algeria|Africa| Africa|OFF-TEN-10000025|Office Supplies|     Storage| Tenex Lock

In [116]:
# Convert Unix timestamp to timestamp
df3=sdf9B.select(
    from_unixtime(col("order_date").alias("timestamp_1")),
    from_unixtime(col("order_date"),"MM-dd-yyyy HH:mm:ss").alias("timestamp_2")
  )
df3.printSchema()
df3.show(truncate=False)

root
 |-- from_unixtime(order_date AS timestamp_1, yyyy-MM-dd HH:mm:ss): string (nullable = true)
 |-- timestamp_2: string (nullable = true)

+-------------------------------------------------------------+----------------------+
|from_unixtime(order_date AS timestamp_1, yyyy-MM-dd HH:mm:ss)|timestamp_2           |
+-------------------------------------------------------------+----------------------+
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|+122939-01-14 09:35:59                                       |01-14-+122939 09:35:59|
|-61925-10-18 17:26:51                                        |10-18--61925

In [119]:
# Convert Unix timestamp to timestamp
sdf9B.select(from_unixtime(sdf9B['order_date']/1000000000).alias('order_date')).show()
#sdf9B.select(from_unixtime(sdf9B['order_date']/1000000000)).show()

+-------------------+
|         order_date|
+-------------------+
|2011-01-01 00:00:00|
|2011-01-01 00:00:00|
|2011-01-01 00:00:00|
|2011-01-01 00:00:00|
|2011-01-01 00:00:00|
|2011-01-01 00:00:00|
|2011-01-02 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
|2011-01-03 00:00:00|
+-------------------+
only showing top 20 rows



In [120]:
test = sdf9B.\
    withColumn('order_date',from_unixtime(col('order_date')/1000000000))
test.printSchema()
test.show(3)

root
 |-- order_id: string (nullable = true)
 |-- order_date: string (nullable = true)
 |-- ship_date: long (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- discount: double (nullable = true)
 |-- profit: double (nullable = true)
 |-- shipping_cost: double (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- year: string (nullable = true)

+-------------+-------------------+-------------------+--------------+---------------+--------+---------------+---------+------+-------+------------

In [105]:
test = sdf9B.\
    withColumn('order_date',from_unixtime(col('order_date')/1000000000)).\
    withColumn('order_date',to_date(col('order_date'), "y-M-d H:m:s")).\
    withColumn('ship_date',from_unixtime(col('ship_date')/1000000000)).\
    withColumn('ship_date',to_date(col('ship_date'), "y-M-d H:m:s"))
test.printSchema()
test.show()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- discount: double (nullable = true)
 |-- profit: double (nullable = true)
 |-- shipping_cost: double (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- year: string (nullable = true)

+---------------+----------+----------+--------------+-----------------+-----------+-----------------+--------------+------+--------------+-----------

In [121]:
# Date Time Patterns https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html
#
sdf9B = sdf9B.\
    withColumn('order_date',from_unixtime(col('order_date')/1000000000)).\
    withColumn('order_date',to_date(col('order_date'), "y-M-d H:m:s")).\
    withColumn('ship_date',from_unixtime(col('ship_date')/1000000000)).\
    withColumn('ship_date',to_date(col('ship_date'), "y-M-d H:m:s"))
sdf9B.printSchema()
sdf9B.show()

root
 |-- order_id: string (nullable = true)
 |-- order_date: date (nullable = true)
 |-- ship_date: date (nullable = true)
 |-- ship_mode: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- segment: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market: string (nullable = true)
 |-- region: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales: long (nullable = true)
 |-- quantity: long (nullable = true)
 |-- discount: double (nullable = true)
 |-- profit: double (nullable = true)
 |-- shipping_cost: double (nullable = true)
 |-- order_priority: string (nullable = true)
 |-- year: string (nullable = true)

+---------------+----------+----------+--------------+-----------------+-----------+-----------------+--------------+------+--------------+-----------

In [122]:
# Reduced Data
#selCols = ['Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit']
selCols = ['year','segment','region','category','sales','quantity','profit']
#df9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select('Order ID','Order Date','Segment','Region','Category','Sales','Quantity','Profit')
sdf9C = spark9.read.parquet('Orders2.parquet',inferSchema=True,header = True, quote = '"').select(selCols)
#df9.dtypes
sdf9C.dtypes

[('year', 'string'),
 ('segment', 'string'),
 ('region', 'string'),
 ('category', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('profit', 'double')]

In [ ]:
# remove , and " from data
#import pandas as pd
#p_df = pd.read_csv('SSdatafile.csv')
#p_df = pd.read_csv('datafile.csv')
#p_df2 = p_df.replace(',', ' ', regex=True)
#p_df3 = p_df2.replace('"', '', regex=True)
#p_df3.to_csv('SSdatafile2.csv',index=False,sep=',')
#!head SSdatafile2.csv

##Spark SQL

In [123]:
spark9.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [124]:
#air_quality_sdf.createOrReplaceTempView("air_quality_sdf")
sdf9C.createOrReplaceTempView("orders_temp")

In [125]:
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
|         |orders_temp|       true|
+---------+-----------+-----------+



In [126]:
spark9.sql("desc orders_temp").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    year|   string|   NULL|
| segment|   string|   NULL|
|  region|   string|   NULL|
|category|   string|   NULL|
|   sales|   bigint|   NULL|
|quantity|   bigint|   NULL|
|  profit|   double|   NULL|
+--------+---------+-------+



In [127]:
spark9.sql('select * from Orders_Temp limit 5').show()

+----+-----------+-------+---------------+-----+--------+-------+
|year|    segment| region|       category|sales|quantity| profit|
+----+-----------+-------+---------------+-----+--------+-------+
|2011|   Consumer| Africa|Office Supplies|  408|       2| 106.14|
|2011|   Consumer|Oceania|Office Supplies|  120|       3| 36.036|
|2011|   Consumer|   EMEA|Office Supplies|   66|       4|  29.64|
|2011|Home Office|  North|Office Supplies|   45|       3|-26.055|
|2011|   Consumer|Oceania|      Furniture|  114|       5|  37.77|
+----+-----------+-------+---------------+-----+--------+-------+



In [128]:
sql_drop_table = """
drop table if exists praxisdb.sh_orders
"""

sql_drop_database = """
drop database if exists praxisdb cascade
"""

sql_create_database = """
create database if not exists praxisdb
location '/content/praxisdb/'
"""

sql_create_table = """
create table if not exists praxisdb.sh_orders
using parquet
as select  * from Orders_Temp
"""

#as select to_timestamp(date) as date_parsed, *

print("dropping database...")
result_drop_db = spark9.sql(sql_drop_database)

print("creating database...")
result_create_db = spark9.sql(sql_create_database)

print("dropping table...")
result_droptable = spark9.sql(sql_drop_table)

print("creating table...")
result_create_table = spark9.sql(sql_create_table)

dropping database...
creating database...
dropping table...
creating table...


In [129]:
spark9.sql("use praxisdb")
spark9.sql("show tables").show()

+---------+-----------+-----------+
|namespace|  tableName|isTemporary|
+---------+-----------+-----------+
| praxisdb|  sh_orders|      false|
|         |orders_temp|       true|
+---------+-----------+-----------+



In [130]:
spark9.sql("select * from praxisdb.sh_orders").show(5)
#.select("date_parsed", "O_3").show(5)

+----+-----------+-------+---------------+-----+--------+-------+
|year|    segment| region|       category|sales|quantity| profit|
+----+-----------+-------+---------------+-----+--------+-------+
|2011|   Consumer| Africa|Office Supplies|  408|       2| 106.14|
|2011|   Consumer|Oceania|Office Supplies|  120|       3| 36.036|
|2011|   Consumer|   EMEA|Office Supplies|   66|       4|  29.64|
|2011|Home Office|  North|Office Supplies|   45|       3|-26.055|
|2011|   Consumer|Oceania|      Furniture|  114|       5|  37.77|
+----+-----------+-------+---------------+-----+--------+-------+
only showing top 5 rows



In [131]:
spark9.sql("select distinct(region) from praxisdb.sh_orders").show()

+--------------+
|        region|
+--------------+
|        Africa|
|    North Asia|
|  Central Asia|
|         South|
|     Caribbean|
|       Central|
|          East|
|       Oceania|
|        Canada|
|          West|
|          EMEA|
|Southeast Asia|
|         North|
+--------------+



##CrossTab

###Inner Query

In [132]:
sums_df = spark9.sql("""
    SELECT
        Segment,
        SUM(IF(Region='Africa',Sales,0)) As Africa,
        SUM(IF(Region='North Asia',Sales,0)) As North_Asia,
        SUM(IF(Region='Central Asia',Sales,0)) As Central_Asia,
        SUM(IF(Region='Caribbean',Sales,0)) As Caribbean,
        SUM(IF(Region='Oceania',Sales,0)) As Oceania,
        SUM(IF(Region='Canada',Sales,0)) As Canada,
        SUM(IF(Region='EMEA',Sales,0)) As EMEA,
        SUM(IF(Region='South East Asia',Sales,0)) As SE_Asia,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='North',Sales,0)) As North,
        SUM(IF(Region='South',Sales,0)) As South
    FROM praxisdb.sh_orders
        GROUP BY Segment WITH ROLLUP
""")
sums_df.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+
|    Segment|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  North|  South|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 209996| 280016|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 394220| 496029|
|       NULL|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1248192|1600960|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 643976| 824915|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+-------+



###Outer Query

In [133]:
#sums_df.registerTempTable("sums")
sums_df.createOrReplaceTempView("sums_T")
#CrossTab = sqlContext.sql("""
CrossTab = spark9.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  Africa,North_Asia,Central_Asia,Caribbean,
  Oceania,Canada,EMEA,SE_Asia,
  Central,East,West,South,
  Africa +North_Asia + Central_Asia+ Caribbean + Oceania + Canada+EMEA + SE_Asia + Central + East + West + South Totals
FROM sums_t
order by Dims desc
""")
CrossTab.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|       Dims|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  South|  Totals|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 280016| 1930632|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 496029| 3176232|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 824915| 5403411|
|    #Totals|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1600960|10510275|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+



###Using Subquery

In [134]:
CrossTab2 = spark9.sql("""
 SELECT
  IFNULL(Segment,'#Totals') Dims,
  Africa,North_Asia,Central_Asia,Caribbean,
  Oceania,Canada,EMEA,SE_Asia,
  Central,East,West,South,
  Africa +North_Asia + Central_Asia+ Caribbean + Oceania + Canada+EMEA + SE_Asia + Central + East + West + South Totals
FROM (
SELECT
        Segment,
        SUM(IF(Region='Africa',Sales,0)) As Africa,
        SUM(IF(Region='North Asia',Sales,0)) As North_Asia,
        SUM(IF(Region='Central Asia',Sales,0)) As Central_Asia,
        SUM(IF(Region='Caribbean',Sales,0)) As Caribbean,
        SUM(IF(Region='Oceania',Sales,0)) As Oceania,
        SUM(IF(Region='Canada',Sales,0)) As Canada,
        SUM(IF(Region='EMEA',Sales,0)) As EMEA,
        SUM(IF(Region='South East Asia',Sales,0)) As SE_Asia,
        SUM(IF(Region='Central',Sales,0)) As Central,
        SUM(IF(Region='East',Sales,0)) As East,
        SUM(IF(Region='West',Sales,0)) As West,
        SUM(IF(Region='North',Sales,0)) As North,
        SUM(IF(Region='South',Sales,0)) As South
    FROM praxisdb.sh_orders
        GROUP BY Segment WITH ROLLUP
    )
as sums
order by Dims desc
""")
CrossTab2.show()

+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|       Dims|Africa|North_Asia|Central_Asia|Caribbean|Oceania|Canada|  EMEA|SE_Asia|Central|  East|  West|  South|  Totals|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+
|Home Office|155077|    175177|      148235|    57395| 197812| 11896|148862|      0| 491961|127465|136736| 280016| 1930632|
|  Corporate|204948|    265762|      235552|   104535| 322831| 19319|250559|      0| 850405|200422|225870| 496029| 3176232|
|   Consumer|423751|    407410|      369052|   162351| 579564| 35717|406763|      0|1480033|350947|362908| 824915| 5403411|
|    #Totals|783776|    848349|      752839|   324281|1100207| 66932|806184|      0|2822399|678834|725514|1600960|10510275|
+-----------+------+----------+------------+---------+-------+------+------+-------+-------+------+------+-------+--------+



In [135]:
spark9.sql("select distinct(year) from praxisdb.sh_orders").show()

+----+
|year|
+----+
|2012|
|2014|
|2013|
|2011|
+----+



In [136]:
CrossTab2 = spark9.sql("""
 SELECT
  IFNULL(Category,'#Totals') Dims,
  Y2011,Y2012,Y2013,Y2014,Y2011+Y2012+Y2013+Y2014 Totals
FROM (
SELECT
        Category,
        SUM(IF(Year='2011',Sales,0)) As Y2011,
        SUM(IF(Year='2012',Sales,0)) As Y2012,
        SUM(IF(Year='2013',Sales,0)) As Y2013,
        SUM(IF(Year='2014',Sales,0)) As Y2014
    FROM praxisdb.sh_orders
        GROUP BY Category WITH ROLLUP
    )
as sums
order by Dims desc
""")
CrossTab2.show()

+---------------+-------+-------+-------+-------+--------+
|           Dims|  Y2011|  Y2012|  Y2013|  Y2014|  Totals|
+---------------+-------+-------+-------+-------+--------+
|     Technology| 827698|1023467|1277328|1616198| 4744691|
|Office Supplies| 675642| 795113|1010793|1305782| 3787330|
|      Furniture| 756171| 858913|1117739|1378061| 4110884|
|        #Totals|2259511|2677493|3405860|4300041|12642905|
+---------------+-------+-------+-------+-------+--------+



#Parquet to Pandas again

In [137]:
!ls /content/praxisdb/sh_orders

part-00000-bd4d2673-7709-48ad-8787-f4e5deaac456-c000.snappy.parquet  _SUCCESS


In [138]:
df7 = pd.read_parquet('/content/praxisdb/sh_orders/part-00000-bd4d2673-7709-48ad-8787-f4e5deaac456-c000.snappy.parquet')
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      51290 non-null  object 
 1   segment   51290 non-null  object 
 2   region    51290 non-null  object 
 3   category  51290 non-null  object 
 4   sales     51290 non-null  int64  
 5   quantity  51290 non-null  int64  
 6   profit    51290 non-null  float64
dtypes: float64(1), int64(2), object(4)
memory usage: 2.7+ MB


#Second Spark

In [139]:
spark39 = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
target_parquet = 'Orders2.parquet'
sdf39B = spark39.read.parquet(target_parquet, inferSchema=True,header = True, quote = '"')
sdf39B.dtypes

[('order_id', 'string'),
 ('order_date', 'bigint'),
 ('ship_date', 'bigint'),
 ('ship_mode', 'string'),
 ('customer_name', 'string'),
 ('segment', 'string'),
 ('state', 'string'),
 ('country', 'string'),
 ('market', 'string'),
 ('region', 'string'),
 ('product_id', 'string'),
 ('category', 'string'),
 ('sub_category', 'string'),
 ('product_name', 'string'),
 ('sales', 'bigint'),
 ('quantity', 'bigint'),
 ('discount', 'double'),
 ('profit', 'double'),
 ('shipping_cost', 'double'),
 ('order_priority', 'string'),
 ('year', 'string')]

In [140]:
sdf39B.repartition(2).write.mode('overwrite').parquet('wpart/Orders239')

In [141]:
spark39.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
| praxisdb|
+---------+



#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself.
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)

# Multiple Spark Contexts
https://medium.com/@achilleus/spark-session-10d0d66d1d24

In [142]:
spark9

In [143]:
spark39